# Разделитьная кластеризация
## Елисеев Е.В.

### Задание

Разработайте программу, которая выполняет кластеризацию заданного набора данных <br>
с помощью алгоритмов k‑Means и k-Medoids. Параметрами программы являются набор данных <br>
 и число кластеров. Программа должна выдавать координаты точек и назначенные им кластера, <br>
  а также значение ошибки кластеризации.
Проведите эксперименты на наборе данных customers (сведения о клиентах банка: скачать  <br>
zip-архив с данными в формате CSV и описанием).  <br>
Выполните визуализацию полученных результатов в следующем виде: <br>
- точечный график, на котором цвет точки отражает принадлежность кластеру;
- зависимость ошибки кластеризации от параметра k. <br>

Доработайте программу, добавив в список ее параметров долю зашумленных объектов <br>
 набора. Дополнительно к ранее реализованным функциям программа должна вносить шум  <br>
 в набор данных: случайным образом изменить заданную долю объектов набора (изменение  <br>
 может заключаться в добавлении/вычитании случайного числа к/из одной/нескольких координат объекта). <br>
Проведите эксперименты на ранее выбранных наборах данных, варьируя долю зашумленных объектов (1%, 3%, 5%, 10%) и <br>
 используя различные значения параметра  (из интервала 3..9). <br>
Выполните визуализацию полученных результатов указанным выше способом. <br>
Подготовьте отчет о выполнении задания и загрузите отчет в формате PDF в систему. <br>
 Отчет должен представлять собой связный и структурированный документ со следующими разделами:   <br>
формулировка задания; 
- гиперссылка на каталог репозитория с исходными текстами, наборами данных и др. сопутствующими материалами; 
- рисунки с результатами визуализации; 
- пояснения, раскрывающие смысл полученных результатов.

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 


data = pd.read_csv('customers.csv')
data

,Row,CustomerId,Age,Education,YearsEmployed,Income,CardDebt,OtherDebt,Defaulted,DebtIncomeRatio
0,0,1,41,2,6,19,0.124,1.073,0.0,6.3
1,1,2,47,1,26,100,4.582,8.218,0.0,12.8
2,2,3,33,2,10,57,6.111,5.802,1.0,20.9
3,3,4,29,2,4,19,0.681,0.516,0.0,6.3
4,4,5,47,1,31,253,9.308,8.908,0.0,7.2
...,...,...,...,...,...,...,...,...,...,...
845,845,846,27,1,5,26,0.548,1.220,NaN,6.8
846,846,847,28,2,7,34,0.359,2.021,0.0,7.0
847,847,848,25,4,0,18,2.802,3.210,1.0,33.4
848,848,849,32,1,12,28,0.116,0.696,0.0,2.9


In [2]:
data = data.drop(columns=['CustomerId', 'Row'])

In [3]:
data.isnull().sum()

Age                  0
Education            0
YearsEmployed        0
Income               0
CardDebt             0
OtherDebt            0
Defaulted          150
DebtIncomeRatio      0
dtype: int64

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


# Разделяем данные на строки, где 'Defaulted' пропущено, и где нет
# train_data = data[data['Defaulted'].notna()]
# test_data = data[data['Defaulted'].isna()]

# # Создаем экземпляр RandomForestClassifier
# clf = RandomForestClassifier()

# # Обучаем модель
# clf.fit(train_data.drop('Defaulted', axis=1), train_data['Defaulted'])

# # Предсказываем пропущенные значения
# test_data['Defaulted'] = clf.predict(test_data.drop('Defaulted', axis=1))

# # Объединяем данные обратно в один DataFrame
# clean_data = pd.concat([train_data, test_data])
# clean_data

data.loc[data['Defaulted'].isnull()] = 0
clean_data = data.copy()

In [5]:
clean_data['Defaulted'].isnull().sum()

0

In [6]:
from sklearn.preprocessing import StandardScaler


df_std = StandardScaler().fit_transform(clean_data)
scaled_df = pd.DataFrame(df_std, index=data.index, columns=clean_data.columns)
scaled_df

,Age,Education,YearsEmployed,Income,CardDebt,OtherDebt,Defaulted,DebtIncomeRatio
0,0.811905,0.544350,-0.132932,-0.492942,-0.575052,-0.451565,-0.523797,-0.293567
1,1.208221,-0.392285,2.794319,1.659051,1.643717,1.780466,-0.523797,0.594010
2,0.283483,0.544350,0.452519,0.516635,2.404708,1.025730,1.909138,1.700067
3,0.019272,0.544350,-0.425657,-0.492942,-0.297830,-0.625566,-0.523797,-0.293567
4,1.208221,-0.392285,3.526132,5.723926,3.995872,1.996015,-0.523797,-0.170672
...,...,...,...,...,...,...,...,...
845,-1.896258,-1.328919,-1.011107,-0.997731,-0.636767,-0.786760,-0.523797,-1.153833
846,-0.046781,0.544350,0.013431,-0.094425,-0.458091,-0.155419,-0.523797,-0.197982
847,-0.244939,2.417619,-1.011107,-0.519510,0.757802,0.216014,1.909138,3.406944
848,0.217430,-0.392285,0.745244,-0.253832,-0.579034,-0.569336,-0.523797,-0.757838


In [7]:
from sklearn.cluster import KMeans
from sklearn_extra.cluster import KMedoids
from sklearn.metrics import silhouette_score
import numpy as np

def cluster_data(data, n_clusters, method:str = 'k-means'):
    point_cluster = {}
    
    if method == 'k-means':
        clasters = KMeans(n_clusters=n_clusters, random_state=37).fit(data)
    else:
        clasters = KMedoids(n_clusters=n_clusters, random_state=37).fit(data)

    labels = clasters.labels_
    error = silhouette_score(data, labels)

    for point, label in zip(data.values, labels):
        point_cluster[tuple(point)] = label

    return point_cluster, error

clasters, error = cluster_data(clean_data, 3, method='kmedoids')
clasters

{(41.0, 2.0, 6.0, 19.0, 0.124, 1.073, 0.0, 6.3): 2,
 (47.0, 1.0, 26.0, 100.0, 4.582, 8.218, 0.0, 12.8): 0,
 (33.0, 2.0, 10.0, 57.0, 6.111000000000001, 5.8020000000000005, 1.0, 20.9): 0,
 (29.0, 2.0, 4.0, 19.0, 0.6809999999999999, 0.516, 0.0, 6.3): 2,
 (47.0, 1.0, 31.0, 253.0, 9.308, 8.908, 0.0, 7.2): 0,
 (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0): 1,
 (38.0, 2.0, 4.0, 56.0, 0.442, 0.4539999999999999, 0.0, 1.6): 0,
 (42.0, 3.0, 0.0, 64.0, 0.2789999999999999, 3.945, 0.0, 6.6): 0,
 (47.0, 3.0, 23.0, 115.0, 0.653, 3.947, 0.0, 4.0): 0,
 (44.0, 3.0, 8.0, 88.0, 0.285, 5.083, 1.0, 6.1): 0,
 (24.0, 1.0, 7.0, 18.0, 0.526, 0.643, 0.0, 6.5): 2,
 (28.0, 3.0, 2.0, 20.0, 0.233, 1.6469999999999998, 1.0, 9.4): 2,
 (29.0, 1.0, 1.0, 17.0, 0.132, 0.293, 0.0, 2.5): 2,
 (43.0, 4.0, 1.0, 26.0, 1.519, 1.237, 0.0, 10.6): 2,
 (29.0, 2.0, 6.0, 25.0, 0.585, 0.465, 0.0, 4.2): 2,
 (36.0, 3.0, 10.0, 43.0, 0.961, 4.629, 0.0, 13.0): 2,
 (28.0, 3.0, 6.0, 47.0, 5.574, 3.732, 1.0, 19.8): 2,
 (45.0, 1.0, 19.0, 77.0, 2.303, 

In [8]:
error

0.541550071486601